In [1]:
from pathlib import Path
from typing import List
import gzip
import numpy as np
import pandas as pd

import torch.utils.data as Data 
import torch
import matplotlib.pyplot as plt
from torch import nn ,optim

In [2]:
if torch.cuda.is_available():
    device=torch.device('cuda')
    print("GPU获取")

GPU获取


In [3]:
def create_data_iter(data,target):
    target_code=[]
    for i in target:
        if(i=="F"):
            target_code.append(1)
        else:
            target_code.append(0)
    target_code_array=np.array(target_code)
    train_target=torch.tensor(target_code_array,dtype=torch.float)
    spatial_map=torch.tensor(data, dtype=torch.float)
    batch_size=5
    dataset_test= Data.TensorDataset(spatial_map, train_target)
    data_iter= Data.DataLoader(dataset_test, batch_size, shuffle=True)
    return data_iter

In [4]:
data_participants_dir='/input/participants1.tsv'#获取全部实验者的数据
data_participants_tsv=pd.read_csv(data_participants_dir,sep='\t')
data_NoNaN=data_participants_tsv.dropna(axis=0,how='any')#去除NAN无效数据
data_sex=data_NoNaN.sex.head(100)
data_sex_code=[]


In [5]:
spatial_map_trian = np.load('/input/spatial_map_train.npy100')#获取样本数据
len(spatial_map_trian)


100

In [6]:
data_iter=create_data_iter(spatial_map_trian,data_sex)

In [1]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv=nn.Sequential(
            nn.Conv3d(1,4,12,4),#240*240*240->58*58*58
            nn.BatchNorm3d(4),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#58*58*58->29*29*29
            nn.Conv3d(4,8,3,1,1),#29*29*29->29*29*29
            nn.BatchNorm3d(8),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#29*29*29->14*14*14
            nn.Conv3d(8,16,3,1,1),#14*14*14->14*14*14
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(2,2),#14*14*14->7*7*7
            nn.Conv3d(16,32,3,1,1),#7*7*7->7*7*7
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(2,2)#7*7*7->3*3*3
            
        )
        self.fc=nn.Sequential(
            nn.Linear(32*3*3*3,512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64,2)#预测性别  
         )
    def forward (self,img):
        padding=nn.ConstantPad3d((10,10,0,0,0,0),0.0)
        img=padding(img)
        batch_size, x, y, z = img.shape
        img = img.reshape(batch_size, 1, x, y, z)
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

net=AlexNet()


NameError: name 'nn' is not defined

In [ ]:
net=net.to(device)
loss=nn.CrossEntropyLoss()
result=[]
lr=0.001
num_epoches=30
optimizer=torch.optim.Adam(net.parameters(),lr=lr)
for epoch in range(num_epoches):
    train_l_sum, train_acc_sum, n=0.0,0.0,0
    for X,y in data_iter:
        X=X.to(device)
        y=y.to(device)
        y=y.long()
        y_hat=net(X)
        l=loss(y_hat,y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_l_sum+=l.cpu().item()
        n+=y.shape[0]
        train_acc_sum += (y_hat.argmax(dim=1) ==y).sum().cpu().item()
    result.append(train_acc_sum / n)
    print('epoch %d, loss %.4f,  train acc %.3f'%(epoch + 1, train_l_sum / n,train_acc_sum / n))

In [ ]:
data_sex2=data_NoNaN.sex.tail(100)

In [ ]:
spatial_map_test = np.load('/input/spatial_map_test.npy')#获取样本数据


In [ ]:
data_iter_test=create_data_iter(spatial_map_test,data_sex2)
    

In [ ]:
data_participants_dir='/input/participants2.tsv'#获取全部实验者的数据
data_participants_tsv2=pd.read_csv(data_participants_dir,sep='\t')
data_NoNaN2=data_participants_tsv2.dropna(axis=0,how='any')#去除NAN无效数据
data_sex2=data_NoNaN2.sex.tail(100)


In [ ]:
spatial_map_vaild= np.load('/input/spatial_map_2vaild.npy')#获取样本数据
len(spatial_map_vaild)


In [ ]:
del data_iter#32GB内存的电脑会出现内存不足  先把data-iter验证

In [2]:
data_iter_vaild=create_data_iter(spatial_map_vaild,data_sex2)

In [ ]:
def evaluate_acc(data,net):
    test_acc_sum,n=0.0,0
    for X,y in data:
        
        X=X.to(device)
        y=y.to(device)
        y=y.long()
        y_hat=net(X)
        test_acc_sum += (y_hat.argmax(dim=1) ==y).sum().cpu().item()
        n+=y.shape[0]
    return test_acc_sum / n

In [ ]:
result_train=[]
result_test=[]
for i in range(5):
    result_train.append(evaluate_acc(data_iter,net))
    result_test.append(evaluate_acc(data_iter_test,net))


In [ ]:
result_vaild=[]
for i in range(5):
    result_vaild.append(evaluate_acc(data_iter_vaild,net))

In [ ]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
name_list=['1','2','3','4','5']
width=0.4
x =list(range(len(name_list)))  
plt.plot(x, result_train, marker='o', mec='r', mfc='w',label='PIOP1train')
plt.plot(x, result_test, marker='*', ms=10,label='PI0P1test')
plt.plot(x, result_vaild, marker='^', ms=10,label='PI0P2test')
plt.legend()  # 让图例生效